In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    model='llama-3.1-70b-versatile',
    temperature=0.5,
    max_tokens=None
)

In [11]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader('https://jobs.nike.com/job/R-33460')
page_data = loader.load().pop().page_content
print(page_data)



Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsFIND A STOREBECOME A MEMBERSIGN UP FOR EMAILSEND US FEEDBACKSTUDENT DISCOUNTSGET HELPGET HELPOrder StatusDeliveryReturnsPayment OptionsC

In [12]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)
chain_extract = prompt_template | llm
response = chain_extract.invoke(input={'page_data'})

In [13]:
print(response.content)

```json
[
  {
    "role": "Software Engineer",
    "experience": "3-5 years",
    "skills": ["Java", "Spring Boot", "MySQL"],
    "description": "We are looking for a highly skilled Software Engineer to join our team. The ideal candidate will have a strong background in Java and experience with Spring Boot and MySQL."
  },
  {
    "role": "Data Scientist",
    "experience": "2-4 years",
    "skills": ["Python", "Machine Learning", "Data Visualization"],
    "description": "We are seeking a talented Data Scientist to join our team. The ideal candidate will have a strong background in Python and experience with machine learning and data visualization."
  },
  {
    "role": "DevOps Engineer",
    "experience": "4-6 years",
    "skills": ["AWS", "Docker", "Kubernetes"],
    "description": "We are looking for a highly skilled DevOps Engineer to join our team. The ideal candidate will have a strong background in AWS and experience with Docker and Kubernetes."
  }
]
```


In [7]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_resp = json_parser.parse(response.content)
json_resp

[{'role': 'Software Engineer',
  'experience': '3-5 years',
  'skills': ['Java', 'Spring', 'Hibernate'],
  'description': 'We are seeking a highly skilled Software Engineer to join our team. The ideal candidate will have a strong background in Java, Spring, and Hibernate. The successful candidate will be responsible for designing, developing, and testing software applications.'},
 {'role': 'Data Scientist',
  'experience': '2-3 years',
  'skills': ['Python', 'Machine Learning', 'Data Analysis'],
  'description': 'We are looking for a talented Data Scientist to join our team. The ideal candidate will have a strong background in Python, Machine Learning, and Data Analysis. The successful candidate will be responsible for analyzing complex data sets and developing predictive models.'},
 {'role': 'DevOps Engineer',
  'experience': '4-6 years',
  'skills': ['AWS', 'Docker', 'Kubernetes'],
  'description': 'We are seeking a highly skilled DevOps Engineer to join our team. The ideal candidate

In [8]:
import pandas as pd
df = pd.read_csv('my_portfolio.csv')
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [9]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row['Techstack'],
                       metadatas={'links':row['Links']},
                       ids=[str(uuid.uuid4())]
                       )

In [17]:
result = collection.query(query_texts=['Experience in Python','Experience in React.js'],n_results=2).get('metadatas',[])
result

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [10]:
job=json_resp
job

[{'role': 'Software Engineer',
  'experience': '3-5 years',
  'skills': ['Java', 'Spring', 'Hibernate'],
  'description': 'We are seeking a highly skilled Software Engineer to join our team. The ideal candidate will have a strong background in Java, Spring, and Hibernate. The successful candidate will be responsible for designing, developing, and testing software applications.'},
 {'role': 'Data Scientist',
  'experience': '2-3 years',
  'skills': ['Python', 'Machine Learning', 'Data Analysis'],
  'description': 'We are looking for a talented Data Scientist to join our team. The ideal candidate will have a strong background in Python, Machine Learning, and Data Analysis. The successful candidate will be responsible for analyzing complex data sets and developing predictive models.'},
 {'role': 'DevOps Engineer',
  'experience': '4-6 years',
  'skills': ['AWS', 'Docker', 'Kubernetes'],
  'description': 'We are seeking a highly skilled DevOps Engineer to join our team. The ideal candidate

In [ ]:
# result = []
# for i in range(len(job)):
#     query_result = collection.query(query_texts=job[i]['skills'], n_results=2)
#     result.append(query_result.get('metadatas', []))
# result

[[[{'links': 'https://example.com/java-portfolio'},
   {'links': 'https://example.com/android-portfolio'}],
  [{'links': 'https://example.com/java-portfolio'},
   {'links': 'https://example.com/kotlin-backend-portfolio'}],
  [{'links': 'https://example.com/magento-portfolio'},
   {'links': 'https://example.com/wordpress-portfolio'}]],
 [[{'links': 'https://example.com/ml-python-portfolio'},
   {'links': 'https://example.com/python-portfolio'}],
  [{'links': 'https://example.com/ml-python-portfolio'},
   {'links': 'https://example.com/wordpress-portfolio'}],
  [{'links': 'https://example.com/magento-portfolio'},
   {'links': 'https://example.com/wordpress-portfolio'}]],
 [[{'links': 'https://example.com/ios-ar-portfolio'},
   {'links': 'https://example.com/wordpress-portfolio'}],
  [{'links': 'https://example.com/devops-portfolio'},
   {'links': 'https://example.com/ios-ar-portfolio'}],
  [{'links': 'https://example.com/devops-portfolio'},
   {'links': 'https://example.com/kotlin-backen

In [37]:
links = collection.query(query_texts=job[0]['skills'],n_results=2).get('metadatas',[])

In [41]:
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}]]

In [42]:
job[0]['skills']

['Java', 'Spring Boot', 'SQL']

In [29]:
job[1]['skills']

['Python', 'Machine Learning', 'Data Visualization']

In [30]:
job[2]['skills']

['AWS', 'Docker', 'Kubernetes']

In [39]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({'job_description':str(job),'link_list':links})
print(res.content)

Subject: Expert Software Solutions for Your Business Needs

Dear [Client],

I hope this email finds you well. I came across your job postings for a Software Engineer, Data Scientist, and DevOps Engineer, and I'm excited to introduce you to AtliQ, an AI & Software Consulting company that can help you fulfill your software development needs.

At AtliQ, we have a team of highly skilled professionals with expertise in designing, developing, and testing software applications. Our capabilities align perfectly with the requirements of your job postings. We have extensive experience in Java, Spring Boot, and SQL, which are essential skills for your Software Engineer role. Additionally, our team has expertise in Python, Machine Learning, and Data Visualization, making us an ideal fit for your Data Scientist position. Our DevOps Engineers are well-versed in AWS, Docker, and Kubernetes, ensuring the smooth operation of your software systems.

Our company has a proven track record of delivering ta